### Load tensorflow

In [1]:
import tensorflow as tf

C:\Users\bananth\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

In [16]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import model_selection
#from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline
from importlib import reload
import warnings
warnings.filterwarnings('ignore')

### Collect Data (prices.csv)

In [4]:
data = pd.read_csv('prices.csv')#, encoding = 'unicode_escape')

In [9]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [6]:
data.shape

(851264, 7)

### Check all columns in the dataset

In [7]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Drop columns `date` and  `symbol`

In [11]:
data.drop(['date', 'symbol'], axis=1, inplace=True)

In [12]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [17]:
data1 = data.copy(deep=True)

In [18]:
data1=data.head(1000)

In [19]:
data1.shape

(1000, 5)

### Divide the data into train and test sets

In [34]:
x=data1[['open','low','high','volume']]
y=data1["close"]

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [38]:
x_test.shape

(300, 4)

In [39]:
x_train.shape

(700, 4)

## Building the graph in tensorflow

1.Define input data placeholders

In [60]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

#Normalize the data
#x_n = tf.layers.batch_normalization(x,training=True)
x_n = tf.nn.l2_normalize(x,1)
#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [61]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

3.Prediction

In [62]:
#We will use normalized data
y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [63]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [64]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [65]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [66]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:x_train, y_:y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  7614.526
Training loss at step:  10  is  3438.6426
Training loss at step:  20  is  3114.7883
Training loss at step:  30  is  3089.7612
Training loss at step:  40  is  3087.7058
Training loss at step:  50  is  3087.6328
Training loss at step:  60  is  3087.555
Training loss at step:  70  is  3087.5247
Training loss at step:  80  is  3087.6018
Training loss at step:  90  is  3087.594


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [67]:
shape_w=tf.shape(W)

sess.run(W)

array([[2.6261436e-03],
       [2.6021248e-03],
       [2.6471072e-03],
       [3.3643013e+01]], dtype=float32)

In [68]:
shape_b=tf.shape(b)

sess.run(b)

array([33.64303], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [69]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [88]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, test_loss = sess.run([train_op,loss],feed_dict={x:x_test, y_:y_test})
    
    if epoch % 10 == 0:
        print ('Test loss at step: ', epoch, ' is ', test_loss)

Test loss at step:  0  is  4135.2
Test loss at step:  10  is  4135.2
Test loss at step:  20  is  4135.2
Test loss at step:  30  is  4135.2
Test loss at step:  40  is  4135.2
Test loss at step:  50  is  4135.2
Test loss at step:  60  is  4135.2
Test loss at step:  70  is  4135.2
Test loss at step:  80  is  4135.2
Test loss at step:  90  is  4135.2


In [87]:
print(train_loss-test_loss)

-1047.6099


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [89]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape((784,),input_shape=(28,28,)))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(1))

#Comile the model
model.compile(optimizer='sgd', loss='mse', metrics=['accuracy'])

### Execute the model

In [90]:
model.fit(x_train, y_train, 
          validation_data=(x_test, y_test), 
          epochs=100,
          batch_size=x_train.shape[0])

Train on 700 samples, validate on 300 samples
Epoch 1/100
700/700 [==============================] - 0s 442us/step - loss: 7505.8872 - acc: 0.0000e+00 - val_loss: 162.1730 - val_acc: 0.0000e+00
Epoch 2/100
700/700 [==============================] - 0s 3us/step - loss: 6352.6357 - acc: 0.0000e+00 - val_loss: 60830.6055 - val_acc: 0.0000e+00
Epoch 3/100
700/700 [==============================] - 0s 6us/step - loss: 1874.9443 - acc: 0.0000e+00 - val_loss: 651299.3125 - val_acc: 0.0000e+00
Epoch 4/100
700/700 [==============================] - 0s 6us/step - loss: 3224.3979 - acc: 0.0000e+00 - val_loss: 9096.6934 - val_acc: 0.0000e+00
Epoch 5/100
700/700 [==============================] - 0s 5us/step - loss: 7001.5972 - acc: 0.0000e+00 - val_loss: 3502.0867 - val_acc: 0.0000e+00
Epoch 6/100
700/700 [==============================] - 0s 9us/step - loss: 5224.5220 - acc: 0.0000e+00 - val_loss: 42097.6250 - val_acc: 0.0000e+00
Epoch 7/100
700/700 [==============================] - 0s 9us/step 

Epoch 57/100
700/700 [==============================] - 0s 4us/step - loss: 711.6043 - acc: 0.0000e+00 - val_loss: 8654.2471 - val_acc: 0.0000e+00
Epoch 58/100
700/700 [==============================] - 0s 6us/step - loss: 2457.1206 - acc: 0.0000e+00 - val_loss: 861.5908 - val_acc: 0.0000e+00
Epoch 59/100
700/700 [==============================] - 0s 4us/step - loss: 4904.0200 - acc: 0.0000e+00 - val_loss: 1182.6897 - val_acc: 0.0000e+00
Epoch 60/100
700/700 [==============================] - 0s 4us/step - loss: 2873.7495 - acc: 0.0000e+00 - val_loss: 56771.6875 - val_acc: 0.0000e+00
Epoch 61/100
700/700 [==============================] - 0s 6us/step - loss: 7916.6685 - acc: 0.0014 - val_loss: 12887.8799 - val_acc: 0.0000e+00
Epoch 62/100
700/700 [==============================] - 0s 6us/step - loss: 3304.3667 - acc: 0.0000e+00 - val_loss: 3655.1516 - val_acc: 0.0000e+00
Epoch 63/100
700/700 [==============================] - 0s 6us/step - loss: 3253.0686 - acc: 0.0000e+00 - val_loss: 

### Build a 2 layer neural network with input layer, 5 neurons in first hidden layer and one neuron in the output layer.

In [91]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape((784,),input_shape=(28,28,)))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

model.add(tf.keras.layers.Dense(5))

model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='sgd', loss='mse', metrics=['accuracy'])

In [92]:
model.fit(x_train, y_train, 
          validation_data=(x_test, y_test), 
          epochs=100,
          batch_size=x_train.shape[0])

Train on 700 samples, validate on 300 samples
Epoch 1/100
700/700 [==============================] - 0s 660us/step - loss: 7933.3823 - acc: 0.0000e+00 - val_loss: 9435.1738 - val_acc: 0.0000e+00
Epoch 2/100
700/700 [==============================] - 0s 7us/step - loss: 7907.3384 - acc: 0.0000e+00 - val_loss: 13840.4668 - val_acc: 0.0000e+00
Epoch 3/100
700/700 [==============================] - 0s 6us/step - loss: 7365.5693 - acc: 0.0000e+00 - val_loss: 96266720.0000 - val_acc: 0.0000e+00
Epoch 4/100
700/700 [==============================] - 0s 7us/step - loss: 2564400.2500 - acc: 0.0000e+00 - val_loss: 206705080789944928043008.0000 - val_acc: 0.0000e+00
Epoch 5/100
700/700 [==============================] - 0s 6us/step - loss: 8017732520909836648448.0000 - acc: 0.0000e+00 - val_loss: inf - val_acc: 0.0000e+00
Epoch 6/100
700/700 [==============================] - 0s 10us/step - loss: inf - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 7/100
700/700 [====================

Epoch 61/100
700/700 [==============================] - 0s 5us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 62/100
700/700 [==============================] - 0s 4us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 63/100
700/700 [==============================] - 0s 6us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 64/100
700/700 [==============================] - 0s 6us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 65/100
700/700 [==============================] - 0s 5us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 66/100
700/700 [==============================] - 0s 6us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 67/100
700/700 [==============================] - 0s 6us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 68/100
700/700 [==========================

#### Compare the accuracy scores across all the 3 models.

## All three accuracy scores are comparable and hence possibly inaccurate